In [2]:
import os
import json
import numpy as np
import PIL
from PIL import Image
from PIL import ImageDraw

In [5]:
train_img_path = 'data/AERIAL-CD/data/1.Training/원천데이터'
val_img_path = 'data/AERIAL-CD/data/2.Validation/원천데이터'

train_json_path = 'data/AERIAL-CD/data/1.Training/라벨링데이터'
val_json_path = 'data/AERIAL-CD/data/2.Validation/라벨링데이터'

train_label_path = 'data/AERIAL-CD/train/label'
val_label_path = 'data/AERIAL-CD/val/label'

train_before_img_path = 'data/AERIAL-CD/train/A'
val_before_img_path = 'data/AERIAL-CD/val/A'

train_after_img_path = 'data/AERIAL-CD/train/B'
val_after_img_path = 'data/AERIAL-CD/val/B'

os.makedirs(train_label_path, exist_ok=True)
os.makedirs(val_label_path, exist_ok=True)
os.makedirs(train_before_img_path, exist_ok=True)
os.makedirs(val_before_img_path, exist_ok=True)
os.makedirs(train_after_img_path, exist_ok=True)
os.makedirs(val_after_img_path, exist_ok=True)

In [7]:
def create_binary_image_with_polygon(poly_points, img_width, img_height):
    image_size = (img_width, img_height)
    image_array = np.zeros(image_size, dtype=np.uint8)

    scaled_polygons = []
    for poly in poly_points:
        for i, (x, y) in enumerate(poly):
            if x <= img_height:
                scaled_polygons.append([(int(poly_point[0]) + img_height, int(poly_point[1])) for poly_point in poly])
            else :
                scaled_polygons.append([(int(poly_point[0]), int(poly_point[1])) for poly_point in poly])

    image = Image.new("L", image_size, color=0)
    draw = ImageDraw.Draw(image)

    for scaled_polygon in scaled_polygons:
        draw.polygon(scaled_polygon, outline=1, fill=1)

    binary_image = np.array(image)
    before_label = binary_image[:, :img_height]
    after_label = binary_image[:, img_height:]

    return after_label

In [5]:
# train 
train_img_list = os.listdir(train_img_path)
train_json_list = os.listdir(train_json_path)

print(len(train_img_list))
print(len(train_json_list))

for i, train_json in enumerate(train_json_list):

    train_json_ = os.path.join(train_json_path, train_json)
    with open(train_json_, 'r') as f:

        json_data = json.load(f)

        img_id = json_data['info']['info.id']
        img_width = json_data['images']['images.width']
        img_height = json_data['images']['images.height']

        polygons = json_data['annotations']
        poly_points = []
        for poly in polygons:
            poly_points.append(poly['polygon.points'])
        label = create_binary_image_with_polygon(poly_points, img_width, img_height)

    tiff_file_path = os.path.join(train_img_path, train_img_list[i])
    img_tiff = Image.open(tiff_file_path)
    numpy_array = np.array(img_tiff)

    before =  numpy_array[:, :img_height, :]
    after = numpy_array[:, img_height:, :]

    before_img = Image.fromarray(before)
    after_img = Image.fromarray(after)
    label_img = Image.fromarray(label * 255)

    before_img.save(os.path.join(train_before_img_path, 'train_{}.png'.format(i)))
    after_img.save(os.path.join(train_after_img_path, 'train_{}.png'.format(i)))
    label_img.save(os.path.join(train_label_path, 'train_{}.png'.format(i)))

40623
40623


In [8]:
# Val 
val_img_list = os.listdir(val_img_path)
val_json_list = os.listdir(val_json_path)

print(len(val_img_list))
print(len(val_json_list))

for i, val_json in enumerate(val_json_list):

    val_json_ = os.path.join(val_json_path, val_json)
    with open(val_json_, 'r') as f:

        json_data = json.load(f)

        img_id = json_data['info']['info.id']
        img_width = json_data['images']['images.width']
        img_height = json_data['images']['images.height']

        polygons = json_data['annotations']
        poly_points = []
        for poly in polygons:
            poly_points.append(poly['polygon.points'])
        label = create_binary_image_with_polygon(poly_points, img_width, img_height)

    tiff_file_path = os.path.join(val_img_path, val_img_list[i])
    img_tiff = Image.open(tiff_file_path)
    numpy_array = np.array(img_tiff)

    before =  numpy_array[:, :img_height, :]
    after = numpy_array[:, img_height:, :]

    before_img = Image.fromarray(before)
    after_img = Image.fromarray(after)
    label_img = Image.fromarray(label * 255)

    before_img.save(os.path.join(val_before_img_path, 'val_{}.png'.format(i)))
    after_img.save(os.path.join(val_after_img_path, 'val_{}.png'.format(i)))
    label_img.save(os.path.join(val_label_path, 'val_{}.png'.format(i)))

5078
5078
